In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "C:/Users/Maitr/Desktop/accuracy check/new data/train"
test_dir = "C:/Users/Maitr/Desktop/accuracy check/new data/Test_Rescaled"

train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.1
)

test_datagen = ImageDataGenerator()  

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 4448 images belonging to 4 classes.
Found 1846 images belonging to 4 classes.


In [4]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in base.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dense(256, activation='relu')(x)
x = Dense(4, activation='softmax')(x)

model = Model(inputs=base.input, outputs=x)

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0005),
    metrics=['accuracy']
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding2d                │ (None, 230, 230, 3)       │               0 │ input_layer[0][0]          │
│ (ZeroPadding2D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 112, 112, 64)      │           9,408 │ zero_padding2d[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 112, 112, 64)      │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 112, 112, 64)      │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding2d_1              │ (None, 114, 114, 64)      │               0 │ conv1_relu[0][0]           │
│ (ZeroPadding2D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1 (MaxPooling2D)          │ (None, 56, 56, 64)        │               0 │ zero_padding2d_1[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_bn             │ (None, 56, 56, 64)        │             256 │ pool1[0][0]                │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_0_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 56, 56, 128)       │           8,192 │ conv2_block1_0_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 56, 56, 128)       │             512 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 56, 56, 128)       │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 56, 56, 32)        │          36,864 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_concat           │ (None, 56, 56, 96)        │               0 │ pool1[0][0],               │
│ (Concatenate)                 │                           │               

 Total params: 7,300,932 (27.85 MB)

 Trainable params: 263,428 (1.00 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [5]:
history = model.fit(
    train_gen,
    epochs=25,
    validation_data=test_gen
)


C:\Users\Maitr\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.6502 - loss: 1.1236 - val_accuracy: 0.6419 - val_loss: 1.0239
Epoch 2/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.7808 - loss: 0.5799 - val_accuracy: 0.7021 - val_loss: 0.8882
Epoch 3/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - accuracy: 0.8159 - loss: 0.5164 - val_accuracy: 0.7839 - val_loss: 0.5642
Epoch 4/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step - accuracy: 0.8359 - loss: 0.4395 - val_accuracy: 0.8126 - val_loss: 0.5333
Epoch 5/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.8368 - loss: 0.4440 - val_accuracy: 0.6912 - val_loss: 0.9128
Epoch 6/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.8647 - loss: 0.3684 - val_accuracy: 0.8629 - val_loss: 0.3985
Epoch 7/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.8671 - loss: 0.3433 - val_accuracy: 0.7698 - val_loss: 0.6569
Epoch 8/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.8730 - loss: 0.3328 - val_accu

In [6]:
model.save("fungal_model_densenet121.h5")


In [7]:
loss, acc = model.evaluate(test_gen)
print("Test Accuracy:", acc)


58/58 ━━━━━━━━━━━━━━━━━━━━ 44s 755ms/step - accuracy: 0.8922 - loss: 0.3753
Test Accuracy: 0.8921993374824524


In [8]:

train_acc = history.history['accuracy']
print("Training Accuracy:", train_acc[-1])


Training Accuracy: 0.931654691696167
